In [1]:
pip install --upgrade pip setuptools wheel

In [3]:
pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [7]:
from PyPDF2 import PdfReader

def text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text
    
        
path = "TempDocs.pdf"
pdf_data = text_from_pdf(path)
print(pdf_data)

I the undersigned declare, confirm and agree as folows: 1. Al the particulars/details filed 
in/provided during the journey of opening the InstaSave account (Particulars) have been 
entered by me/us. I verify and confirm that al the information and Particulars 
entered/recorded by me are true and correct and I have not withheld any information 
required for opening the Sav ings Account. 2. The Particulars read along with the Terms 
(hereinafter defined) shal together constitute the Savings Account opening documents and 
shal be referred to as “electronic application form” or “EAF”. 3. I understand that my Savings 
Account openi ng request is subject to verification and if any discrepancy in the information/ 
documentation provided is noted as per the Bank policy and/or applicable law, the Bank has 
a right to reject the EAF, account opening request and/or close/freeze my Savings Ac count 
and retain the Savings Account opening documents. 4. I agree and undertake to provide any 
further

In [9]:
#chunking
chunks = [pdf_data[i:i+500] for i in range(0,len(pdf_data),1000)]
print(len(chunks))
print(chunks[0])

15
I the undersigned declare, confirm and agree as folows: 1. Al the particulars/details filed 
in/provided during the journey of opening the InstaSave account (Particulars) have been 
entered by me/us. I verify and confirm that al the information and Particulars 
entered/recorded by me are true and correct and I have not withheld any information 
required for opening the Sav ings Account. 2. The Particulars read along with the Terms 
(hereinafter defined) shal together constitute the Savings Accou


In [11]:
!pip install requests


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Users\MAHESH\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [15]:
def call_ollama(prompt,model="gemma3"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model":model,
        "prompt":prompt,
        "temprature":0.0,
        "stream":False
    }
    response = requests.post(url,json=payload)
    data = response.json()
    return data["response"]

In [17]:
import requests
summaries = []
for idx, chunk in enumerate(chunks):
    prompt = f"Summarize the text into 50 words\n\n{chunk}"
    summary = call_ollama(prompt)
    summaries.append({
        "chunk_id":idx+1,
        "instruction":f"Summarize the text in the part{idx+1}",
        "input":chunk,
        "output":f"Summary of the part{idx+1}: {summary} " 
    })


In [19]:
print(len(summaries))

15


In [21]:
import json
import os
with open("ReqDocs.json","w",encoding="utf-8") as f:
    for item in summaries:
        f.write(json.dumps(item,ensure_ascii=False)+"\n")
    

In [23]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

class PDFDataset(Dataset):
    def __init__(self,data,tokenizer,max_input_length=1024,max_output_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        item = self.data[idx]
        inputs = self.tokenizer(
            item['instruction'] + "\n" + item["input"],
            truncation = True,
            padding = "max_length",
            max_length = self.max_input_length,
            return_tensors = "pt"
        )
        outputs = self.tokenizer(
            item["output"],
            truncation=True,
            padding="max_length",
            max_length=self.max_output_length,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs.input_ids.squeeze(),
            "attention_mask": inputs.attention_mask.squeeze(),
            "labels": outputs.input_ids.squeeze()
        }

        
pdf_dataset = PDFDataset(summaries, tokenizer)
print("✅ PyTorch Dataset ready. Number of samples:", len(pdf_dataset))


✅ PyTorch Dataset ready. Number of samples: 15


In [25]:
print(pdf_dataset.data)

[{'chunk_id': 1, 'instruction': 'Summarize the text in the part1', 'input': 'I the undersigned declare, confirm and agree as folows: 1. Al the particulars/details filed \nin/provided during the journey of opening the InstaSave account (Particulars) have been \nentered by me/us. I verify and confirm that al the information and Particulars \nentered/recorded by me are true and correct and I have not withheld any information \nrequired for opening the Sav ings Account. 2. The Particulars read along with the Terms \n(hereinafter defined) shal together constitute the Savings Accou', 'output': 'Summary of the part1: Here’s a 50-word summary of the text:\n\nThis document confirms the accuracy of information provided during the opening of an InstaSave account. The signatory affirms all details entered are true, complete, and not withheld. The provided particulars, alongside the account’s terms, collectively form the agreement for the savings account. '}, {'chunk_id': 2, 'instruction': 'Summari

In [27]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

training_args = TrainingArguments(
    output_dir="./GENAI_01P_finetuned-flan_model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",
    logging_strategy="steps",     # log by steps (not only by epoch)
    logging_steps=4,  
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = pdf_dataset
)

trainer.train()

#Save the fine_tuned model
model.save_pretrained("./GENAI_01P_finetuned-flan_model")
tokenizer.save_pretrained("./GENAI_01P_finetuned-flan_model")
print("✅ Fine-tuning complete and model saved")





C:\Users\MAHESH\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
4,19.108900
8,19.390200


✅ Fine-tuning complete and model saved


In [29]:
from transformers import pipeline
qa = pipeline("text2text-generation",model="./GENAI_01P_finetuned-flan_model",tokenizer="./GENAI_01P_finetuned-flan_model")

Device set to use cpu


In [31]:
document = """
Artificial intelligence (AI) is a rapidly evolving field of computer science
that focuses on the development of systems capable of performing tasks that
typically require human intelligence. These tasks include learning, reasoning,
problem-solving, perception, and language understanding.
"""

# Combine document and instruction
prompt = f"{document}\n\nSummarize the above document in 3 bullet points."
result = qa(prompt, max_length=128)
print(result[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Artificial intelligence (AI) is a rapidly evolving field of computer science that focuses on the development of systems capable of performing tasks that typically require human intelligence.


In [33]:
document = """
Essay: The Airplane Crash on Sunday 
Last Sunday, an unfortunate airplane crash left many people in shock and sorrow. 
The aircraft, which was on a routine journey, suddenly met with trouble and could not land safely.
Within moments, lives were lost, and families were left grieving. Rescue teams and medical staff immediately reached the spot to save survivors and control the situation. 
Airplane crashes are rare, but when they happen, they remind us of the importance of strict safety rules in aviation.
Experts will now investigate the cause of this accident, which could be due to weather, a technical fault, or human error.
Their findings will help improve air travel safety in the future. 
This tragic event also shows the courage of emergency workers who risk their lives to help others.
While the accident brought sadness, it also reminds us that every lesson learned can save countless lives in the years to come. 
We must honor the victims by making air travel safer and more reliable. 
"""

# Combine document and instruction
prompt = f"{document}\n\nSummarize the above document in 3 bullet points."
result = qa(prompt, max_length=128)
print(result[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Airplane crashes are rare, but when they happen, they remind us of the importance of strict safety rules in aviation.


In [35]:
docs = """
Rahul: Hey! Did you watch the football match last night?
Aman: Of course! What a game it was. The last-minute goal was unbelievable.
Rahul: I know, right? I thought it was going to end in a draw.
Aman: Same here. But that free-kick completely changed the game.
Rahul: The crowd went crazy. I wish we were there at the stadium.
Aman: Next time, let’s get tickets and watch it live together.
Rahul: Deal! It’ll be so much fun. 
"""
prompt = f"{docs}\n\nSummarize the above chat"
result = qa(prompt,max_length=128)
print(result[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Aman and Rahul watched the football match last night. Aman and Rahul are going to watch it live together.


In [37]:
new_docs = """
In Natural Language Processing, Tokenization is the process of breaking given text into individual words.
Assuming that given document of text input contains paragraphs, it could broken down to sentences or words. NLTK provides tokenization at two levels: word level and sentence leve
"""
prompt = f"{new_docs}\n\nSummarize the above chat"
result = qa(prompt,max_length=128)
print(result[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


NLTK provides tokenization at two levels: word level and sentence leve


In [49]:
pip install gradio

  Using cached gradio-5.47.2-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.13.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached ruff-0.13.2-py3-none-win_amd64.whl.metadata (26 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached typer-0.19.2-py3-none-any.whl.metadata (16 kB)
Using cached gradio-5.47.2-py3-none-any.whl (60.4 MB)
Using cached gradio_client-1.13.3-py3-none-any.whl (325 kB)
Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
Using cached groovy-0.1.2-py3-none-any.whl (14 kB)
Using cached safehttpx-0.1.6-py3-none-any.whl (8.7 k

In [53]:
import gradio as gr
from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


def chunk_pdf_text(pdf_path, chunk_size=400, max_chunks=10):
    reader = PdfReader(pdf_path)
    pdf_text = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            pdf_text += text + "\n"
    chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
    return chunks[:max_chunks]


def summarize_pdf(file):
    chunks = chunk_pdf_text(file.name)
    summaries = []
    for i, chunk in enumerate(chunks):
        prompt = f"Summarize this section (chunk {i+1}): {chunk}"
        result = qa_pipeline(prompt, max_new_tokens=100)[0]['generated_text']
        summaries.append(f"Chunk {i+1} summary: {result}")
    return "\n\n".join(summaries)


iface = gr.Interface(
    fn=summarize_pdf,
    inputs=gr.File(file_types=[".pdf"]),
    outputs=gr.Textbox(lines=20, placeholder="PDF summaries will appear here..."),
    title="PDF Summarizer",
    description="Upload a PDF, the model will summarize each chunk of the document."
)


iface.launch()

Device set to use cpu


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
